<a href="https://colab.research.google.com/github/kahunahana/MGMT-5900-_Capstone_Brainiacs-6/blob/main/Mock_Data_FakerJS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-cloud-bigquery
# !pip install apache-beam

import os
from google.cloud import bigquery

from google.colab import auth
auth.authenticate_user()

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from google.cloud import storage
import json
import numpy as np

import random
from datetime import datetime, timezone

# To supress warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
project_id = 'thelook-product-recom-1'
client = bigquery.Client(project=project_id)

In [ ]:
# get order_items
query = """
SELECT *
FROM `thelook-product-recom-1.theLook.order_items`
"""

# Run the query and load the result into a Pandas DataFrame
df_order_items = client.query(query).to_dataframe()


In [ ]:
# get products
query = """
SELECT *
FROM `thelook-product-recom-1.theLook.products`
"""

# Run the query and load the result into a Pandas DataFrame
df_products = client.query(query).to_dataframe()


In [ ]:
# get orders
query = """
SELECT *
FROM `thelook-product-recom-1.theLook.orders`
"""

# Run the query and load the result into a Pandas DataFrame
df_orders = client.query(query).to_dataframe()


In [ ]:
# get users
query = """
SELECT *
FROM `thelook-product-recom-1.theLook.users`
"""

# Run the query and load the result into a Pandas DataFrame
df_users = client.query(query).to_dataframe()


In [ ]:
list(df_orders['status'].unique())

['Shipped', 'Complete', 'Returned', 'Cancelled', 'Processing']

In [ ]:
# Create a datetime object for the given date and time in UTC timezone
current_time = datetime.now()

# Format the time in the desired format
formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S UTC')


def generate_orders():
  genders = ['M', 'F', 'Unknown']
  order_id = df_orders.order_id.to_list()
  current_max_id = int(max(order_id) + 1) # Make sure that the new id is 101
  status_choice = list(df_orders['status'].unique())
  # Add the new id to the user_ids so that the new id, 101 can be used as the starting point for the new id and the next time code runs we get 102
  order_id.append(current_max_id)
  blank_or_1 = ['',1]
  users_id = df_users.id.to_list()
  #print(type(users_id))

  return {
        "order_id" : current_max_id,
        "user_id" : int(random.choice(users_id)),
        "status" : random.choice(status_choice),
        "gender": random.choice(genders),
        "create_at" : formatted_time,
        "returned_at" : random.choice(blank_or_1),
        "shipped_at" : random.choice(blank_or_1),
        "delivered_at" : random.choice(blank_or_1),
        "num_of_items": random.randint(1,10)

    }

In [ ]:
generate_orders()

{'order_id': 125282,
 'user_id': 37083,
 'status': 'Processing',
 'gender': 'M',
 'create_at': '2023-08-15 22:34:17 UTC',
 'returned_at': 1,
 'shipped_at': '',
 'delivered_at': '',
 'num_of_items': 3}

In [ ]:
def generate_order_items():

  order_ids = df_order_items.id.to_list()


  order_details = generate_orders()
  order_id = order_details['order_id']
  user_id = order_details['user_id']
  num_of_items = order_details['num_of_items']
  ordered_at = order_details['create_at']

  product_ids = df_products.id.to_list()

  product_id = random.choice(product_ids)

  statuses = list(df_order_items['status'].unique())
  blank_or_1 = ['',1]

  sales_price = df_products.loc[df_products['id'] == product_id, 'retail_price'].iloc[0]

  # Create an empty dict
  objs = {}

  for i in range(1, num_of_items+1) :

    current_max_id = int(max(order_ids) + 1)
    order_ids.append(current_max_id)

    objs[i] = {
            "id" : int(current_max_id),
            "order_id" : int(order_id),
            "user_id" : int(user_id),
            "product_id" : int(random.choice(product_ids)),
            "inventory_id": random.randint(1000,9999),
            "status" : random.choice(statuses),
            "ordered_at" : ordered_at,
            "returned_at" : random.choice(blank_or_1),
            "shipped_at" : random.choice(blank_or_1),
            "sale_price": sales_price
      }

  json_data = objs.values()
  json_data = json.dumps(json_data, indent=1)

  return json_data

In [ ]:
generate_order_items()

'{\n "1": {\n  "id": 181823,\n  "order_id": 125282,\n  "user_id": 24180,\n  "product_id": 24579,\n  "inventory_id": 8893,\n  "status": "Processing",\n  "ordered_at": "2023-08-15 23:43:12 UTC",\n  "returned_at": "",\n  "shipped_at": "",\n  "sale_price": 14.329999923706056\n },\n "2": {\n  "id": 181824,\n  "order_id": 125282,\n  "user_id": 24180,\n  "product_id": 13814,\n  "inventory_id": 8228,\n  "status": "Processing",\n  "ordered_at": "2023-08-15 23:43:12 UTC",\n  "returned_at": 1,\n  "shipped_at": "",\n  "sale_price": 14.329999923706056\n },\n "3": {\n  "id": 181825,\n  "order_id": 125282,\n  "user_id": 24180,\n  "product_id": 5014,\n  "inventory_id": 5052,\n  "status": "Processing",\n  "ordered_at": "2023-08-15 23:43:12 UTC",\n  "returned_at": 1,\n  "shipped_at": "",\n  "sale_price": 14.329999923706056\n }\n}'

In [ ]:
# 2. Store Synthetic Data in Cloud Storage (CS) Bucket
bucket_name = 'mock_public_data'
local_file_path = '/content/orders.json' # Local path where the file will be saved
blob_name = 'orders.json'

client = storage.Client()
bucket = client.get_bucket(bucket_name)

# Delete all existing files in the bucket
blobs = bucket.list_blobs()
for blob in blobs:
    blob.delete()

# Generate data
data = [generate_orders() for _ in range(1)] #loop through and save to bucket

# Save the data to a local file
with open(local_file_path, 'w') as file:
    json.dump(data, file)

# Upload the new file to the bucket
blob = bucket.blob(blob_name)
blob.upload_from_filename(local_file_path)

print(f"File {local_file_path} uploaded to {blob_name}.")

File /content/orders.json uploaded to orders.json.


In [ ]:
from google.cloud import storage

def delete_blobs(bucket_name, prefix=None):
    """Deletes all blobs in the bucket that match the prefix."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)

    for blob in blobs:
        blob.delete()
        print(f"Blob {blob.name} deleted.")

def upload_file_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    with open(source_file_name, 'r') as file:
        blob.upload_from_file(file)

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# Example usage
bucket_name = "mock_public_data"
prefix = "https://console.cloud.google.com/storage/browser/mock_public_data" # Optional: specify a prefix to delete only files that match
source_file_name = "/content/orders.json"
destination_blob_name = "orders.json"

delete_blobs(bucket_name, prefix)
upload_file_to_gcs(bucket_name, source_file_name, destination_blob_name)


Blob https://console.cloud.google.com/storage/browser/mock_public_data/orders.json deleted.
File /content/orders.json uploaded to orders.json.


In [ ]:
# # # 3. Set Up Dataflow to Read from Cloud Storage and Write to Pub/Sub
# # import apache_beam as beam


# pipeline_options = beam.options.pipeline_options.PipelineOptions()
# with beam.Pipeline(options=pipeline_options) as p:
#     # Read from Cloud Storage
#     lines = p | 'Read from GCS' >> beam.io.ReadFromText('gs://mock_public_data/mock_data.json')

#     # Process the data as needed
#     processed = lines | 'Process Data' >> beam.Map(lambda x: x)  # Modify as needed

#     # Write to Pub/Sub
#     processed | 'Write to Pub/Sub' >> beam.io.WriteToPubSub(f'projects/{project_id}/topics/{your-topic-id}')

In [ ]:
# 4. Simulate Streaming Data into Pub/Sub
from google.cloud import pubsub_v1
import timae
import json

project_id = "your-project-id"
topic_id = "your-topic-id"

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_id)

while True:
    # Create a synthetic message
    message = {"key": "value"}  # Adjust this to match your data structure
    message_json = json.dumps(message)

    # Publish the message
    publisher.publish(topic_path, message_json.encode("utf-8"))

    # Wait for a specified interval
    time.sleep(1)  # Adjust this to control the streaming rate